# Web Scraping de PCPartPicker

Este cuaderno Jupyter contiene un script para extraer datos de compatibilidad y precios de componentes de PC desde `es.pcpartpicker.com`. El objetivo final es generar un dataset en formato **JSONL plano**, ideal para entrenar LLMs.

### Cloudflare
PCPartPicker está fuertemente protegido por el WAF (Web Application Firewall) de Cloudflare. Las librerias convencionales (`requests`, `BeautifulSoup` e incluso Selenium estandar) son bloqueadas instantaneamente. 

**La solucion:**
1. **`undetected_chromedriver`**: Una version modificada de Selenium que oculta las huellas automatizadas del navegador.

2. **Perfil persistente + VPN**: Usando una VPN para evitar bloqueos por IP. Al configurar un "Perfil persistente" en Chrome, el usuario solo necesita resolver el Captcha de Cloudflare una vez manualmente. La cookie de sesion se guarda y el script puede continuar navegando de forma 100% automatizada.

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, NoSuchWindowException
import json
import time
import re
import random
import os

# ================================= CONFIGURACION ====================================
# Limite de paginas. Usa `float('inf')` para recorrer todo el catalogo.
MAX_MOTHERBOARD_PAGES = float('inf')
MAX_COMPONENT_PAGES = 5

# Archivo de salida estructurado
ARCHIVO_SALIDA = "pcpartpicker_motherboards.jsonl"

# Ruta para el perfil persistente (esta tiene que apuntar a una carpeta creada vacia).
CARPETA_PERFIL = os.path.join(os.path.expanduser("~"), "pcpartpicker_chrome_profile")
# ====================================================================================

## 1. Inicializacion del navegador y gestion de Cloudflare

En esta seccion se define las funciones para arrancar el navegador y manejar los bloqueos.

* **Perfil persistente**: Usamos el argumento `--user-data-dir` para que Chrome guarde las cookies de sesion. Esto es vital para que Cloudflare confie en el bot despues del primer Captcha resuelto manualmente.

* **Gestion de Captchas**: La funcion `comprobar_cloudflare` vigila el titulo de la pestaña. Si detecta "Just a moment", pausa la ejecucion, avisa al usuario por consola y espera pacientemente hasta que el humano resuelva el desafio.

In [ ]:
def iniciar_navegador():
    print("Iniciando navegador...")
    if not os.path.exists(CARPETA_PERFIL):
        os.makedirs(CARPETA_PERFIL)

    options = uc.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    
    # Inyeccion del perfil para conservar la sesion de Cloudflare.
    options.add_argument(f"--user-data-dir={CARPETA_PERFIL}")
    
    driver = uc.Chrome(options=options)
    driver.implicitly_wait(10)
    return driver

# Detecta si Cloudflare pide verificacion manual y pausa el script.
def comprobar_cloudflare(driver):
    """Detecta si Cloudflare pide verificación manual o si hay un Rate Limit."""
    try:
        titulo = driver.title.lower()
        # Detección de Captchas de Cloudflare
        if "just a moment" in titulo or "cloudflare" in titulo or "attention required" in titulo:
            print("\n[!] CLOUDFLARE DETECTADO")
            try:
                WebDriverWait(driver, 120).until_not(EC.title_contains("Just a moment"))
                time.sleep(2)
            except TimeoutException:
                pass
                
        # Deteccion de "Soft-Bans" por demasiadas peticiones (Error 429)
        elif "429" in titulo or "too many requests" in titulo:
            print("\n[!] RATE LIMIT: Espera 60 segundos a que el script se reanude...")
            time.sleep(60)
            driver.refresh() # Recargamos tras la pausa
            time.sleep(5)
            
    except (NoSuchWindowException, WebDriverException):
        raise# Si se cierra la ventana durante el chequeo, pasamos el error arriba.

## 2. Limpieza de datos y extraccion con selectores dinamicos

Aquí es donde resolvemos dos problemas críticos del *scraping*:

1.  **Nomenclatura CSS dinamica**: PCPartPicker usa clases con doble guion bajo (`td__nameWrapper`), lo cual puede romper la busqueda tradicional. Usamos selectores CSS con coincidencia parcial (`[class*='nameWrapper']`) para que el script sea a prueba de balas frente a cambios menores en la web.

2.  **Limpieza de precios (Regex)**: Originalmente, los precios venian contaminados con texto de los botones (`"€160.80Add"`). La funcion `limpiar_precio_a_float` usa expresiones regulares para aislar los numeros y convertirlos a formato `float`, devolviendo `null` si el componente no tiene precio, lo cual es la mejor practica para alimentar un LLM.

In [ ]:
# Espera a que el DOM cargue los productos, simulando pausas humanas.
def esperar_carga_tabla(driver):
    comprobar_cloudflare(driver)
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "tr[class*='tr__product']"))
        )
        time.sleep(random.uniform(1.5, 3.5)) # Pausa aleatoria anti-bot
    except TimeoutException:
        pass

# Espera a que carguen los enlaces de compatibilidad en el perfil de la placa base.
def esperar_pagina_placa(driver):
    comprobar_cloudflare(driver)
    try:
        # Esperamos especificamente un enlace que tenga 'compatible_with='
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='compatible_with=']"))
        )
        time.sleep(random.uniform(1.0, 2.0))
    except TimeoutException:
        pass

# Extrae el numero del `string` y lo convierte a `float` para el LLM.
def limpiar_precio_a_float(texto_bruto):
    if not texto_bruto:
        return None

    # Busca '€' seguido de numeros, ignorando letras pegadas como 'Add'.
    match = re.search(r'€\s*([\d,]+(?:\.\d+)?)', texto_bruto)
    if match:
        try:
            numero_limpio = match.group(1).replace(',', '')
            return float(numero_limpio)
        except ValueError:
            return None
    return None

# Extrae nombres, URLs y precios limpios de la tabla actual.
def extraer_items_de_pagina(driver):
    items = []
    try:
        filas = driver.find_elements(By.CSS_SELECTOR, "tr[class*='tr__product']")
        for fila in filas:
            try:
                nombre = fila.find_element(By.CSS_SELECTOR, "[class*='nameWrapper'] p").text.strip()
                
                try:
                    url = fila.find_element(By.CSS_SELECTOR, "td[class*='name'] a").get_attribute("href")
                except:
                    url = None

                try:
                    precio_elemento = fila.find_element(By.CSS_SELECTOR, "[class*='price']")
                    precio_float = limpiar_precio_a_float(precio_elemento.text.strip())
                except:
                    precio_float = None

                items.append({"nombre": nombre, "precio": precio_float, "url": url})
            except:
                continue 
    except (NoSuchWindowException, WebDriverException):
        raise 
    except Exception as e:
        print(f"[!] Error extrayendo items: {e}")
    return items

## 3. Aplanamiento de datos (Flattening) y paginacion

Para que un LLM entienda las relaciones entre componentes sin confundirse con estructuras JSON anidadas y complejas, se ha implementado el **aplanamiento de datos** (igual que en Pangoly).

En lugar de crear un objeto masivo por cada placa base, la funcion `raspar_y_guardar_componentes` genera una relacion 1 a 1 por cada linea (placa base - componente compatible) y la guarda inmediatamente en el archivo `.jsonl`.

In [ ]:
def guardar_en_jsonl(datos, archivo):
    with open(archivo, 'a', encoding='utf-8') as f:
        f.write(json.dumps(datos, ensure_ascii=False) + '\n')

# Navega por las categorias y guarda relaciones planas (1 a 1) en el JSONL.
def raspar_y_guardar_componentes(driver, cat_nombre, url_categoria, mb_nombre, mb_precio):
    pagina_actual = 1
    
    while pagina_actual <= MAX_COMPONENT_PAGES:
        print(f"Scraping pagina {pagina_actual} de categoria...")
        try:
            # Navegacion mediante fragmentos de URL (Hashes).
            driver.get(f"{url_categoria}#page={pagina_actual}")
            esperar_carga_tabla(driver)
            
            items = extraer_items_de_pagina(driver)
            if not items:
                break
                
            # APLANAMIENTO: Creacion del esquema estructurado para el LLM
            for comp in items:
                relacion_plana = {
                    "motherboard": mb_nombre,
                    "motherboard_price": mb_precio,
                    "currency": "EUR",
                    "component_type": cat_nombre.lower(),
                    "component_name": comp["nombre"],
                    "component_price": comp["precio"],
                    "compatible": True
                }
                guardar_en_jsonl(relacion_plana, ARCHIVO_SALIDA)
                
            pagina_actual += 1
        except (NoSuchWindowException, WebDriverException):
            raise # Propagamos el error si el usuario cierra la ventana.
        except Exception:
            break

## 4. Ejecucion principal

La funcion `main` coordina todo el proceso. Itera sobre el catalogo de placas base, entra en cada perfil y extrae los enlaces de compatibilidad.

**Manejo de cierres de ventana:**
Uno de los mayores retos fue evitar que el script colapsara con un error masivo de codigo (`Traceback`) si el usuario decidia cerrar la ventana de Chrome manualmente para detener el proceso. 
Se ha implementado un sistema que captura excepciones como `NoSuchWindowException` o errores de sesion inválida (`InvalidSessionIdException`). Si esto ocurre, el script asume que es un cierre intencionado, finaliza la ejecucion ordenadamente y asegura que todos los datos recopilados hasta ese milisegundo estén a salvo.

In [ ]:
def main():
    driver = None
    try:
        driver = iniciar_navegador()
        url_base_motherboards = "https://es.pcpartpicker.com/products/motherboard/"

        # PAGINA POR LA QUE EMPEZAR
        pagina_motherboard = 2
        # =========================

        print("\nIniciando recoleccion de placas base...")
        
        while pagina_motherboard <= MAX_MOTHERBOARD_PAGES:
            print(f"\nScraping catalogo de placas base - pagina {pagina_motherboard}")
            driver.get(f"{url_base_motherboards}#page={pagina_motherboard}")
            esperar_carga_tabla(driver)
            
            motherboards = extraer_items_de_pagina(driver)
            if not motherboards:
                break
            
            for mb in motherboards:
                if not mb["url"]: continue
                
                mb_nombre = mb['nombre']
                mb_precio = mb['precio']
                print(f"\nProcesando: {mb_nombre}...")
                
                try:
                    driver.get(mb["url"])
                    esperar_pagina_placa(driver) # Usamos la nueva espera rápida y precisa
                    
                    try:
                        # ESTRATEGIA INFALIBLE: Buscar cualquier enlace de compatibilidad en toda la web
                        enlaces = driver.find_elements(By.CSS_SELECTOR, "a[href*='compatible_with=']")
                    except (NoSuchWindowException, WebDriverException):
                        raise
                        
                    if not enlaces:
                        print(f"[!] No se encontro lista de compatibilidad para {mb_nombre}.")
                        continue
                    
                    # Extraer y limpiar categorías evitando duplicados
                    categorias_vistas = set()
                    categorias = []
                    for e in enlaces:
                        texto = e.text.strip()
                        url_cat = e.get_attribute("href")
                        
                        if texto and url_cat:
                            nombre_cat_limpio = texto.replace("View Compatible", "").strip()
                            if nombre_cat_limpio not in categorias_vistas:
                                categorias_vistas.add(nombre_cat_limpio)
                                categorias.append({"nombre_categoria": nombre_cat_limpio, "url_categoria": url_cat})
                            
                    for cat in categorias:
                        print(f"\nExtrayendo: {cat['nombre_categoria']}...")
                        raspar_y_guardar_componentes(driver, cat['nombre_categoria'], cat['url_categoria'], mb_nombre, mb_precio)
                        
                except (NoSuchWindowException, WebDriverException):
                    raise
                except Exception:
                    continue
            pagina_motherboard += 1

        print("\nScript completado exitosamente, un saludo socio!")

    # Errores o cierres manuales del navegador
    except KeyboardInterrupt:
        print("\n\n[!] AVISO: El script ha sido detenido manualmente por el usuario.")
        print("[!] Los datos procesados se han guardado en el archivo JSONL.")
        
    except (NoSuchWindowException, WebDriverException) as e:
        # Detectamos si el navegador murió o fue cerrado por el usuario
        error_msg = str(e).lower()
        mensajes_cierre = ["target window already closed", "no such window", "invalid session id", "disconnected", "chrome not reachable"]
        
        if any(msg in error_msg for msg in mensajes_cierre):
            print("\n\n[!] AVISO: Se ha cerrado la ventana del navegador manualmente.")
            print("[!] Los datos procesados se han guardado en el archivo JSONL.")
        else:
            print(f"\n\n[!] Error de WebDriver: {e}")
            
    except Exception as e:
        print(f"\n\n[!] Error inesperado: {e}")
        
    finally:
        if driver:
            try: driver.quit()
            except: pass

# Punto de entrada de ejecución del cuaderno.
if __name__ == "__main__":
    main()